# 연결 설정하기

In [1]:
from sqlalchemy import create_engine
from sqlalchemy import text

engine = create_engine("sqlite+pysqlite:///:memory:", echo=True)

with engine.connect() as conn:
    result = conn.execute(text("SELECT 'hello world'"))
    print(result.all())

2023-12-12 18:16:14,271 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-12 18:16:14,273 INFO sqlalchemy.engine.Engine SELECT 'hello world'
2023-12-12 18:16:14,274 INFO sqlalchemy.engine.Engine [generated in 0.00247s] ()
[('hello world',)]
2023-12-12 18:16:14,275 INFO sqlalchemy.engine.Engine ROLLBACK


# 변경 사항 커밋하기
### Commit as you go

In [2]:
with engine.connect() as conn:
    conn.execute(text("CREATE TABLE some_table (x int, y int)"))
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [
            {"x": 1, "y": 1},
            {"x": 2, "y": 4}
        ]
    )
    conn.commit()

2023-12-12 18:16:36,593 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-12 18:16:36,594 INFO sqlalchemy.engine.Engine CREATE TABLE some_table (x int, y int)
2023-12-12 18:16:36,595 INFO sqlalchemy.engine.Engine [generated in 0.00153s] ()
2023-12-12 18:16:36,596 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2023-12-12 18:16:36,597 INFO sqlalchemy.engine.Engine [generated in 0.00073s] [(1, 1), (2, 4)]
2023-12-12 18:16:36,598 INFO sqlalchemy.engine.Engine COMMIT


### Begin once

In [4]:
with engine.begin() as conn:
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [
            {"x": 6, "y": 8},
            {"x": 9, "y": 10}
        ]
    )

2023-12-12 18:17:11,631 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-12-12 18:17:11,633 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2023-12-12 18:17:11,633 INFO sqlalchemy.engine.Engine [cached since 35.04s ago] [(6, 8), (9, 10)]
2023-12-12 18:17:11,634 INFO sqlalchemy.engine.Engine COMMIT


# SQL 문 실행하기
## 행 가져오기

In [ ]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT x, y FROM some_table"))
    for row in result:
        print(f"x: {row.x} y: {row.y}")